In [2]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

ERROR: Could not find a version that satisfies the requirement statsmodel==0.13.5 (from versions: none)
ERROR: No matching distribution found for statsmodel==0.13.5


In [126]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from script import process_data
from script import geolocaliser
from script import modeling
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [109]:
df = pd.read_csv("data/transactions_with_zone_inondable.csv",encoding="utf-8",sep=";")

C:\Users\flori\AppData\Local\Temp\ipykernel_24492\648495320.py:1: DtypeWarning:

Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.



In [110]:
# Calcul de la distance pour chaque ligne du DataFrame
df['distance_centre_ville'] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_mairie'], row['longitude_mairie']
    ),
    axis=1
)

In [111]:
import ast

# Conversion des chaînes en listes de tuples
df["beach_coordinates"] = df["beach_coordinates"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df["station"] = df["station"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [112]:
def nettoyer_coordinates(coords):
    if isinstance(coords, list):  # Si c'est déjà une liste, la garder
        return coords
    if isinstance(coords, float) or coords is None:  # Si c'est un NaN ou autre, retourner None
        return None
    return coords  # Par défaut, retourner la valeur initiale (si elle est correcte)

df["beach_coordinates"] = df["beach_coordinates"].apply(nettoyer_coordinates)
df["station"] = df["station"].apply(nettoyer_coordinates)

In [113]:
# Calcul de la distance minimale à la plage ou au port en fonction de la disponibilité des données
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["beach_coordinates"])
    if row["beach_coordinates"] else None,  # Si aucune coordonnée plage disponible, retourner None
    axis=1
)

# Si distance_min_beach est NaN, calculer la distance au port
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_port'], row['longitude_port']
    ) if pd.isna(row["distance_min_beach"]) else row["distance_min_beach"],
    axis=1
)


In [114]:
df["distance_min_station"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["station"])
    if row["station"] else None,  # Si aucune coordonnée disponible, retourner None
    axis=1
)

In [115]:
def convertir_en_numerique(df, colonnes):
    """
    Convertit les colonnes spécifiées du DataFrame en type numérique.
    
    Args:
        df (pd.DataFrame): Le DataFrame à modifier.
        colonnes (list): Liste des colonnes à convertir en numériques.
        
    Returns:
        pd.DataFrame: Le DataFrame avec les colonnes converties.
    """
    for col in colonnes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [116]:
# Affichage des modalités de 'code_scenario'
modalites_code_scenario = df['code_scenario'].value_counts()
print(modalites_code_scenario)

code_scenario
02Moy    2617
04Fai    2292
01For    1154
03Mcc     939
Name: count, dtype: int64


In [117]:
# Définir les bornes des tranches et les labels
bins = [0, 10000, 50000, np.inf]  # Définir les bornes des tranches
labels = ['0-10000','10000-50000', 'plus_50000']  # Labels des tranches

# Créer la variable catégorielle 'pop'
df['pop_cut'] = pd.cut(df['Population'], bins=bins, labels=labels, right=False)

# Créer des variables dummies pour les catégories de population
df_dummies_population = pd.get_dummies(df['pop_cut'], prefix='population_',drop_first=True)
df_dummies_population = df_dummies_population.astype(int)

# Ajouter les dummies au DataFrame
df = pd.concat([df, df_dummies_population], axis=1)

In [118]:
# Création des dummies pour la colonne 'code_scenario'
df_dummies_scenario = pd.get_dummies(df['code_scenario'], prefix='scenario')
# df_dummies_scenario = df_dummies_scenario.drop(columns=['scenario_04Fai'])
df_dummies_scenario = df_dummies_scenario.astype(int)

# Ajouter les dummies au DataFrame original
df = pd.concat([df, df_dummies_scenario], axis=1)


In [119]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_communes = pd.get_dummies(df['nom_commune'], prefix='commune')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_communes = df_dummies_communes.drop(columns=['commune_Nice'])

# # Convertir les dummies en entiers (0/1)
df_dummies_communes = df_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_communes], axis=1)

In [120]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_department = pd.get_dummies(df['departmentName'], prefix='department')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_department = df_dummies_department.drop(columns=['department_Alpes-Maritimes'])

# # Convertir les dummies en entiers (0/1)
df_dummies_department = df_dummies_department.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_department], axis=1).drop(columns='departmentName')

In [ ]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)

# Colonnes explicatives, simplifiées à partir de variables_continues
colonnes_explicatives = [
    "log_surface_reelle_bati",
    "log_distance_min_beach",
    # "nombre_pieces_principales",
    "log_distance_min_station",
    "log_distance_centre_ville",
    "zone_inondable",
    # "scenario_01For"
] + [
    col for col in df.columns if col.startswith('population')
] + [
#     col for col in df.columns if col.startswith('scenario')
] + [
#     col for col in df.columns if col.startswith('department')
# ] + [
    col for col in df.columns if col.startswith('commune_')
]

colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements
X,y = modeling.construire_modele_regression(
    df,
    colonnes_explicatives=colonnes_explicatives,
    colonne_dependante=colonne_dependante
)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
# Ajustement avec covariance robuste de type HC3
# model_robust = model.get_robustcov_results(cov_type='HC3')
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Résumé des résultats
# print(model_robust.summary())
# Résultats du modèle
print(model.summary())
print(vif_data)


C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

